# Studi Independen Crawler

## Step 1: Credential Prerequisite

Before you begin using this script. Please create `.env` file from the template `.env.example`. After creating that file, please fill your Kampus Merdeka email and password to the `.env` file.

## Step 2: Installing library

In [ ]:
!pip install python-dotenv pandas requests openpyxl

In [ ]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Step 3: Login to Kampus Merdeka

In [ ]:
config = dotenv_values(".env")
locale.setlocale(locale.LC_ALL, "id")

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        obj = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=obj, headers=headers)
        token = response.json()['data'].get('access_token')
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    return token

token = login()
headers = dict()
headers["Accept"] = "application/json"
headers["Authorization"] = f'Bearer {token}'
headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
print("Login success")

## Step 4: Choose your Kampus Merdeka Studi Independen Activity

In [ ]:
def get_activity_id():
    url =  "https://api.kampusmerdeka.kemdikbud.go.id/mbkm/mahasiswa/activities"
    response = requests.get(url, headers=headers)
    
    activities = response.json().get('data')
    
    print("Your Studi Independen activities")
    for i in range(len(activities)):
        activity = activities[i]["nama_kegiatan"]
        print(f"{i+1}. {activity}")
    input_num = int(input("Choose your activity from number above: "))
    return activities[input_num - 1]["id"]
        
activity_id = get_activity_id()
print(f"Got Activity Id : {activity_id}")

## Step 5: Get Reports

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/studi/report/allweeks/{activity_id}'
response = requests.get(url, headers=headers)
reports = response.json().get('data')

counter_minggu_ke = 1

minggu_tanggal_cols = []
kegiatan_cols = []
hasil_cols = []

for weekly_report in reports[::-1]:
    
    # Handle Minggu Tanggal Cols
    start_date = datetime.strptime(weekly_report["start_date"], "%Y-%m-%dT%H:%M:%SZ")
    end_date = datetime.strptime(weekly_report["end_date"], "%Y-%m-%dT%H:%M:%SZ")
    start_date_str = start_date.strftime("%d %B %Y")
    end_date_str = end_date.strftime("%d %B %Y")
    minggu_tanggal = f"Minggu ke-{counter_minggu_ke} \n({start_date_str} - {end_date_str})"
    minggu_tanggal_cols.append(minggu_tanggal)
    counter_minggu_ke += 1
    
    # Handle Kegiatan Cols
    daily_reports = weekly_report["daily_report"]
    counter = 1
    final_kegiatan_str = ""
    for daily_report in daily_reports:
        report = daily_report["report"]
        final_kegiatan_str += f"{counter}. {report} \n"
        counter += 1
    kegiatan_cols.append(final_kegiatan_str)
    
    # Handle Hasil Cols
    hasil_cols.append(weekly_report["learned_weekly"])

reports = pd.DataFrame({
    "Minggu/Tgl" : minggu_tanggal_cols,
    "Kegiatan Harian": kegiatan_cols,
    "Laporan Mingguan": hasil_cols
})

In [ ]:
reports

## Step 6: Output to Files

In [ ]:
def output_files(df: pd.DataFrame):
    output_types = ["csv", "excel"]
    print("Output Types")
    for i in range(len(output_types)):
        output_type = output_types[i]
        print(f"{i+1}. {output_type}")
    input_num = int(input("Choose your output type from number above: ")) - 1
    
    if input_num == 0:
        df.to_csv("./reports.csv",index=False)
    elif input_num == 1:
        df.to_excel("./reports.xlsx", index=False)
    else:
        raise ValueError("Not a valid input")

    print(f"Creating {output_types[input_num]} with a file name `reports`")
    
output_files(reports)